# VAugmentedReality 

**Progetto di Computer Vision** 

Anno accademico 2020/2021

Simone Scionti

### Cos'è il VAR?
Nel calcio, il VAR (acronimo dell'inglese Video Assistant Referee, che significa "video assistenza arbitrale") è uno strumento usato dai giudici di gara per esaminare situazioni dubbie. I due ufficiali di gara che collaborano con l'arbitro in campo vengono chiamati VAR (un arbitro effettivo, in attività o ritiratosi) e AVAR (assistente del VAR, generalmente un assistente arbitrale). Questi esaminano le situazioni dubbie della partita tramite l'ausilio di filmati. Il metodo di arbitraggio che utilizza tale tecnologia viene comunemente chiamato VAR.

### Il fuorigioco
Tra le situazioni di gioco in cui è l'utilizzo del VAR risulta determinante si trovano i casi di fuorigioco dubbio. Il fuorigioco è una regola del calcio per la quale il giocatore della squadra che attacca, deve essere più distante dalla porta avversaria rispetto agli ultimi due giocatori della squadra che difende. Essendo uno di questi due giocatori spesso il portiere, questa regola nella pratica ha effetto guardando l'ultimo difensore in gioco e l'ultimo attaccante. 



Grazie alla tecnologia VAR, l'arbitro richiede di visionare le immagini dell'azione al fine di individuare "esattamente" la linea immaginaria di fuorigioco al momento del passaggio, in modo da non lasciare dubbi sulla decisione presa, o quasi. 


#### VAR e precisione 
Essendo il VAR una tecnologia basata sull'utilizzo di una videocamera, essa acquisisce un certo numero di immagini per secondo (framerate) per poi mostrarle in sequenza illudendo chi le visualizza sulla continuità della scena. In realtà la continuità percepita aumenta all'aumentare del framerate. 

Solitamente le videocamere utilizzate per il VAR sono a 250 fps (frame per second) il che significa che si ottiene un frame (una immagine) ogni 4 millisecondi. 
Ciò significa che tutto ciò che accade in un ordine di tempo inferiore ai 4 millisecondi, potrebbe non essere "fotografato" dal var.

#### Fuorigioco e precisione del VAR
Secondo alcuni studi, il tempo necessario affinchè la palla si stacchi dal piede del giocatore che effettua il passaggio è di circa 1,2 millisecondi. Questo indica che, in un caso sfortunato, il var potrebbe non aver catturato il frame in cui la palla si è esattamente staccata dal piede, ma averne catturato uno 4 millesimi di secondo prima o dopo. 


Non avere il frame in cui la palla si stacca esattamente dal piede significa non avere il frame esatto in cui bisognerebbe valutare la presenza di fuorigioco, e quindi ci si chiede: Di quanto possono essersi spostati i due giocatori in esame rispetto alla loro posizione nell'ipotetico frame esatto? 

La risposta a questa domanda dipende dalla posizione iniziale dei due giocatori, dalla direzione e la velocità alla quale si muovono.

Essendo la velocità massima delle gambe di un giocatore di 200km/h, supponendo che essi partano dalla stessa posizione all'istante t0 e vadano a quella velocità in direzioni opposte(caso peggiore) si ha un margine di 44cm. 

Questo margine d'errore è facilmente deducibile trasformando **200km/h in 5,55556 cm/ms** e considerando i **4 ms** di distacco tra un frame e l'altro si ottiene **5,55556 * 4 = 22,22 cm.** Moltiplicando questo numero per **2** (i calciatori che si muovono in direzioni opposte) si ottiene un margine d'errore di circa **44cm.**

Ciò significa che nel caso peggiore, un fuorigioco di meno di 44cm non può essere reputato effettivamente fuorigioco. 

### Scopo di questo progetto
Questo progetto nasce con lo scopo di valutare l'esistenza o meno del fuorigioco, in considerazione del margine di errore derivante dal framerate della videocamera utilizzata. Inoltre, piuttosto che utilizzare il "caso peggiore" come riferimento, utilizza i dati di posizione dei giocatori acquisiti direttamente dalla scena. 

#### Tecnologie utilizzate
Per individuare i giocatori all'interno della scena è stata utilizzata la libreria **tensorflow** e nello specifico l'algoritmo **Yolo**. Esso è utilizzato nell'ambito del object detection per via della sua ottima resa in termini di performance. 

Il funzionamento è essenzialmente basato sull'utilizzo di una rete convolutiva che analizza tutte le parti dell’immagine in parallelo, simultaneamente.
Nella versione v3 (l’ultima), YOLO fa predizioni su 3 diverse scale, riducendo l’immagine rispettivamente di 32, 16 e 8, allo scopo di rimanere accurata anche su scale più piccole (le versioni precedenti avevano dei problemi con le immagini piccole).


Il modello di ML è stato costruito attraverso un training set contenente diverse immagini di calciatori. In base al colore della maglia poi, un calciatore viene identificato nella classe 0 o nella classe 1.

## Difficoltà riscontrate
Inizialmente l'applicativo faceva uso esclusivamente di Yolo per l'object detection, ed eseguiva l'algoritmo ad ogni frame. 
Questo ha portato a due grossi problemi:
* prestazioni scarse
* problemi di identificazione dello stesso player tra un frame ed il successivo

Il primo problema era dovuto all'uso di yolo per fare sostanzialmente object tracking, il che non è il suo scopo. 
Il secondo problema era dovuto al fatto che ad ogni frame, rieseguendo l'algoritmo, un object detector non è in grado di fornire informazione sulla "storia" di un oggetto, che è appunto ogni volta un nuovo oggetto rilevato nel frame. 
Per tale motivo è stato inizialmente sviluppato un metodo get_max_error() che facendo uso del principio di località si occupava di associare un player rilevato al frame n con un player rilevato al fame n-1, supponendo che il più vicino (nel frame precedente) fosse lo stesso player nel frame successivo. Grazie a questo metodo veniva poi calcolato il margine d'errore oggetto d'esame. Ciò però non era una vera e propria soluzione. (Il metodo viene lasciato di sotto per scopo illustrativo)

La soluzione a questi problemi è stata utilizzare un Object Tracker. 

###  KCF Object Tracker (Kernalized corelation filter)

KCF è un algoritmo di object tracking con un ottimo compromesso prestazioni-precisione. Inoltre una cosa molto importante è che è molto preciso nella rilevazione di eventuali fallimenti (cosa non del tutto scontata in seguito ad alcuni test con altri algoritmi). 
La rilevazione di un fallimento è molto importante perchè ci permette di riutilizzare yolo per rilevare nuovamente i players in un certo frame ed inizializzare nuovamente i trackers. 
 

## Vecchio metodo get_max_error()
Il metodo **get_max_error** ha lo scopo di calcolare il margine di errore per considerare effettivamente corretto o meno un caso di possibile fuorigioco. 

Nello specifico, il margine di errore viene calcolato in pixel considerando l'effettivo spostamento dei due giocatori in esame nell'arco di un frame. In questo modo, per trattarsi di un fuorigioco certo, un giocatore dovrà trovarsi aldilà della linea di fuorigioco di un numero di pixel maggiore o uguale al margine d'errore calcolato.

Per calcolare l'effettivo distacco tra due giocatori sarebbe necessario conservare la loro precedente posizione, e confrontarla con quella attuale così da capire di quando si sono avvicinati o allontanati. Non avendo però alcuna "etichetta" legata ad un singolo giocatore, il metodo si basa sulla **località** al fine di individuare lo stesso giocatore in due frame conseguenti. 

Infatti è verosimile che dati due frame conseguenti e data la posizione di un giocatore nel primo di essi, nel secondo frame la posizione più vicina a quella salvata sarà molto probabilmente la nuova posizione dello stesso giocatore. 

**Funzionamento del metodo:**
Procedendo inversamente, per ogni frame viene salvata la posizione di tutti i giocatori utili individuati, ed al frame successivo per ogni giocatore si cerca tra le posizioni precedentemente salvate la più vicina a quella corrente, così da procedere ai confronti utili al calcolo del margine d'errore. 


In [ ]:
#declaring constants
framerate = 30 #30 frame per second -> 1/30 = 33 ms between two different consecutive frames.
max_displacement = 20 #between 2 frame we assume that a player might displace for a maximum of 20px in x (abs)

prev_x_coordinates = []

# max error is defined as the difference of the distance of the two adversary
# players between two consequent frames.
def get_max_error(player_x,adversary_x): 
    # first get the previous x coordinate of the two players frames displacement between the two players
    #this method use the locality principle that allow us tu assume that if we found a coordinate really near the current
    # we can consider it as the previous coordinate of the same player.
    
    prev_player_displacement = sys.maxsize
    prev_adversary_displacement = sys.maxsize
    prev_player_x = None
    prev_adversary_x = None
    for x in prev_x_coordinates:
        if(abs(x - player_x) < max_displacement and abs(x - player_x) < prev_player_displacement):
            prev_player_displacement = abs(x - player_x)
            prev_player_x = x
        if(abs(x - adversary_x) < max_displacement and abs(x - adversary_x) < prev_adversary_displacement):
            prev_adversary_displacement = abs(x - adversary_x)
            prev_adversary_x = x
    if prev_player_x is None: #if we did not found the previous position of the player, we assume it did not move.
        prev_player_x = player_x
        #print("found none for player")
    if prev_adversary_x is None: #if we did not found the previous position of the player, we assume it did not move.
        prev_adversary_x = adversary_x
        #print("found none for adversary")
    
    #update prev_x_coordinates for the next time the method will be called
    prev_x_coordinates.append(player_x)
    prev_x_coordinates.append(adversary_x)
    
    #calculate distances between players in the previous step:
    prev_distance_x = abs(prev_player_x - prev_adversary_x)
    distance_x = abs(player_x - adversary_x)
    displacement = abs(prev_distance_x - distance_x) #is the displacement between two frame between a player and it's adversary.
    #print("displacement:" + str(displacement))
    return displacement 

#considerare le coordinate in basso a dx o sx in base alla direzione. 
    

## Vecchio metodo find_useful_players()
Veniva inoltre utilizzato il metodo find_useful_player che, invocato ad ogni frame, si occupava di filtrare i giocatori rilevati nel frame restituendo le coordinate di tutti i giocatori che attaccano, e solo dell'ultimo giocatore che difende ( quelli che servono per il calcolo del fuorigioco ). 
Con l'introduzione del tracker invece tutte sono utili le coordinate di tutti i giocatori, che vengono poi "scremati" (considerato solo l'ultimo che difende) in fase di tracking.
In questo modo se l'ultimo giocatore varia tra un frame ed il successivo, è possibile rilevarlo senza rieseguire l'object detection con Yolo.


In [ ]:
def find_useful_players(players, direction = "left"):
    useful_pl = []
    last_player = None
    #player2 = None
    if direction == "left":
        minx = sys.maxsize
        #min2 = sys.maxsize
    elif direction == "right":
        maxx = -sys.maxsize - 1
        #min2 = sys.minsize
        
    for player in players:
        x, y, w, h, ym = player
        if ym==0:
            if direction is "left": #class 0 are the team that is goint to defend players
                ox,oy = get_real_coordinate(player) #get the minimum in original coordinates
                if ox < minx: 
                    last_player = player
                    minx = ox
            else: #class 0 are the team that is goint to attac players 
                useful_pl.append(player) #save all players
        elif ym==1:
            if direction is "right": #class 1 are the team that is goint to defend players
                ox,oy = get_real_coordinate(player) #get the minimum in original coordinates
                if ox > maxx:
                    last_player = player
                    maxx = ox
            else: #class 1 are the team that is goint to attack players
                useful_pl.append(player) #save all players
    return useful_pl,last_player

# VAgumentedReality with tracker

In [12]:
import sys
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
from numpy import argmax
import imutils
from imutils.video import FPS

In [13]:
def ResizeWithAspectRatio(image, width=None, height=None, inter=cv2.INTER_AREA):
    dim = None
    (h, w) = image.shape[:2]

    if width is None and height is None:
        return image
    if width is None:
        r = height / float(h)
        dim = (int(w * r), height)
    else:
        r = width / float(w)
        dim = (width, int(h * r))

    return cv2.resize(image, dim, interpolation=inter)

In [14]:
model = load_model(r'./Training/model2.h5')
cap = cv2.VideoCapture(r'./data/video.avi')
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
direction = "left"
SCREEN_WIDTH = 1000

In [15]:
def createTracker():
    tracker = cv2.TrackerKCF_create()
    fs_settings = cv2.FileStorage("trackerSettings.yaml",cv2.FILE_STORAGE_READ)
    tracker.read(fs_settings.root())
    return tracker
    

Costruiamo la rete convoluzionale che useremo per fare object detection con la configurazione standard di Yolo. 


In [16]:
net = cv2.dnn.readNet("./files/yolov3.weights", "files/yolov3.cfg")
classes = []

with open("files/coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
    
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

Il metodo **get_players_distinct()** ha lo scopo di restituire, partendo dall'output della rete convoluzionale su un frame, i bounding box che rispettano una certa soglia di confidenza per i players.
Inoltre il metodo identifica, nel caso si tratti di un giocatore, a quale classe appartiene tra quelle con le quali è stato fatto il training. 

In [17]:
def get_players_distinct(outs,height, width):
    class_ids = []
    confidences = []
    boxes = []
    players=[]
    adversaries=[]
    ball = None
    for out in outs:
        print("out")
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.3:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
                
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            if label=='person':
                roi = frame[y:y+h,x:x+w]
            
                try:
                    roi=cv2.resize(roi, (96,96))
                except:
                    continue
                ym=model.predict(np.reshape(roi,(1,96,96,3)))
                ym=argmax(ym)
                if(ym == 0): 
                    if(direction == "right"): # clas 0 are adversaries
                        players.append([x,y,w,h])
                    else:     # clas 1 are adversaries
                        adversaries.append([x,y,w,h])
                elif(ym == 1):     
                    if(direction == "left"):
                        players.append([x,y,w,h])
                    else:
                        adversaries.append([x,y,w,h])
    return players, adversaries

Il metodo **get_real_coordinate** ha lo scopo di restituire le coordinate di un dato player indipendenti dalla prospettiva della camera al fine di valutare il fuorigioco, facendo uso di una **perspectiveTransform**.

La matrice di trasformazione utilizzata è stata costruita facendo riferimento ad un set di punti in un frame del video, ed un set di punti (corrispondenti) in una immagine di un campo da calcio in cui l'asse della camera era ortogonale al campo stesso. 

Grazie al metodo **getPerspectiveTransform(src, dst)** con le coppie di punti raccolte è possibile ottenere una matrice di trasformazione per passare dalle coordinate di un punto nel video (asse della camera parallelo al campo, o quasi) alle coordinate esatte (asse della camera ortogonale al campo).
La matrice permetterà quindi di eseguire non solo una semplice trasformazione affine, ma una trasformazione che più complessa. Infatti la matrice può essere vista come una combinazione di :
* 4 coefficienti che definiscono una trasformazione affine come rotazione, scaling ecc..
* un vettore di traslazione
* un vettore di proiezione

**Documentazione OpenCV:** https://theailearner.com/tag/cv2-getperspectivetransform/

In [18]:
def get_transformation_matrix():
    wi = 1000
    hei = 560
    space = 300
    
    l = space
    r = wi-space
    
    #top left - top right / bottom right - bottom left
    pts1 = np.float32([[0,0],[1000,0],[1000,560],[0,560]]) #vera #TODO: play with 226 and 760 to have a correct one. 
    pts2 = np.float32([[0,0],[wi,0],[r,hei],[l,hei]]) #finta  (seconda tupla 0 ok) (quarta tupla 0 ok)
    return cv2.getPerspectiveTransform(pts1,pts2)

#nuovi
   # pts1 = np.float32([[0,0],[0,560],[1000,1000],[1000,560]]) #vera #TODO: play with 226 and 760 to have a correct one. 
    #pts2 = np.float32([[0,0],[316,560],[1000,1000],[675,560]]) #finta  (seconda tupla 0 ok) (quarta tupla 0 ok)


# pts1 = np.float32([[0,556],[380,40],[1000,556],[600,40]]) #vera #TODO: play with 226 and 760 to have a correct one. 
 #   pts2 = np.float32([[0,556],[300,0],[1000,556],[680,0]]) #finta  (seconda tupla 0 ok) (quarta tupla 0 ok)

def get_real_coordinate(p):
    matrix=np.array(get_transformation_matrix())
    x=p[0]+int(p[2]/2)
    y=p[1]+p[3]
    pts3 = np.float32([[x,y]])
    pts3o=cv2.perspectiveTransform(pts3[None, :, :],matrix)
    x1=int(pts3o[0][0][0])
    y1=int(pts3o[0][0][1])
    return x1,y1

def get_inverse_coordinate_from_point(x,y):
    matrix=image = np.array(get_transformation_matrix())
    inv_matrix = np.linalg.pinv(matrix)
    pts3 = np.float32([[x,y]])
    pts3o=cv2.perspectiveTransform(pts3[None, :, :],inv_matrix)
    x1=int(pts3o[0][0][0])
    y1=int(pts3o[0][0][1])
    #TODO: to remove - only for debug
    if x1 > SCREEN_WIDTH:
        dist_from_center = x1 - (SCREEN_WIDTH/2)
        x1 -= int(0.4 *  dist_from_center)
    else:
        dist_from_center = (SCREEN_WIDTH/2) - x1
        x1 += int(0.4 *  dist_from_center)
    return x1,y1

In [19]:
def get_max_tollerance(prev_player_x,player_x,prev_adversary_x, adversary_x):
    prev_distance_x = abs(prev_player_x - prev_adversary_x)
    distance_x = abs(player_x - adversary_x)
    displacement = abs(prev_distance_x - distance_x) #is the displacement between two frame between a player and it's adversary.
    return displacement 
    

Il metodo **calculate_outside** ha lo scopo di restituire un booleano che indica la presenza di un fuorigioco certo o meno. 

In [20]:
def calculate_outside(player_x,last_adversary_x,error_margin,direction = "left"):
    if direction is "left":
        return (last_adversary_x - player_x) > error_margin  
    else: 
        return (player_x - last_adversary_x) > error_margin

In [21]:
def getPlayersBoundingBoxes():
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    #To identify objects, we are calling net.forward() to get the bounding box and the confidence. 
    outs = net.forward(output_layers)
    return get_players_distinct(outs, height, width)
    #return find_useful_players(playersRecords,direction)

Il segeuente codice fa uso dei metodi definiti al fine di visualizzare a schermo quanto detto. 

In [22]:
adversariesTrackersTuples = []  #each tuple contains the tracker and the last bounding box of the adversary
playersTrackersTuples = [] #each tuple contains the tracker and the last bounding box of the player

while(cap.isOpened()):
    print("opened")
    ret, frame = cap.read()
    if ret is True :
        trackingSuccess = True
        frame = ResizeWithAspectRatio(frame, width=SCREEN_WIDTH)
        gray= cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        height, width, channels = frame.shape
        
        lastAdversaryOldBox = None
        lastAdversaryBox = None
        #Tracker for lastAdversary
        if len(adversariesTrackersTuples) > 0:
            for i in range(len(adversariesTrackersTuples)):
                # grab the new bounding box coordinates of the object
                tracker, oldBox = adversariesTrackersTuples[i]
                (success, newBox) = tracker.update(frame)
                # check to see if the tracking was a success
                if success:
                    if lastAdversaryBox is None : 
                        lastAdversaryBox = newBox
                        lastAdversaryOldBox = oldBox
                    if (direction == "left" and newBox[0] < lastAdversaryBox[0]) or (direction == "right" and newBox[0] > lastAdversaryBox[0]):
                        lastAdversaryBox = newBox
                        lastAdversaryOldBox = oldBox
                        
                    #adversariesTrackersTuples[i] = (tracker,newBox) 
                    #(x, y, w, h) = [int(v) for v in newBox]
                    #cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 255, 0), 2)
                else: 
                    print("lastAdversaryTracker Fail")
                    trackingSuccess = False
            
            #draw the square only for the last adversary player
            if trackingSuccess == True and lastAdversaryBox is not None:
                (x, y, w, h) = [int(v) for v in lastAdversaryBox]
                cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
                print(get_inverse_coordinate_from_point(x, y))
                firstPoint = get_inverse_coordinate_from_point(x, y+400)
                secondPoint = get_inverse_coordinate_from_point(x,y-200)
                cv2.line(frame, firstPoint, secondPoint, (0, 255, 255), 2)
                
               
       
        # we need to use yolo yo find objects and reinit tracker
        else: 
            print("No lastAdversaryTracker")
            trackingSuccess = False
        
        
        #Tracking for all useful players
        if trackingSuccess == True and len(playersTrackersTuples) > 0:
            for i in range(len(playersTrackersTuples)):
                #grab the new bounding box coordinates of the object
                tracker, oldBox = playersTrackersTuples[i]
                (success, newBox) = tracker.update(frame)
                
                # check to see if the tracking was a success
                if success:
                    (x, y, w, h) = [int(v) for v in newBox]
                    player_real_x,player_real_y = get_real_coordinate(newBox)
                    adversary_real_x,adversary_real_y = get_real_coordinate(lastAdversaryBox)
                    #get real coordinates new and old for player
                    #calculate error using real coordinates is the same as we use original coordinates in terms of visual result! 
                    #TODO: need to consider the left bottom or right bottom coordinate of the square based on direction..
                    error = get_max_tollerance(oldBox[0],newBox[0],lastAdversaryOldBox[0], lastAdversaryBox[0])
                    
                    #calculate outsite using real coordinates
                    if calculate_outside(player_real_x,adversary_real_x,error,direction):
                        cv2.rectangle(frame, (x, y), (x + w, y + h), (0,0,255), 2)
                    else:
                        cv2.rectangle(frame, (x, y), (x + w, y + h), (0,255,0), 2)
                    #TODO: draw cm + or -  to outside (how to get cm? I think i need to know the proportions?)
                    #update the last box in the tuple
                    playersTrackersTuples[i] = (tracker,newBox) 
                else: 
                    print("usefulPlayersTracker Fail")
                    trackingSuccess = False 
        else: 
            print("No usefulPlayersTracker")
            trackingSuccess = False
                
         
        if trackingSuccess == False:   
            players, adversaries = getPlayersBoundingBoxes()
            
            #reinit trackers
            adversariesTrackersTuples = []
            for i in range(len(adversaries)):
                x, y, w, h = adversaries[i]
                adversaryBB = (x, y, w, h)
                tracker = createTracker()
                tracker.init(frame, adversaryBB)
                adversariesTrackersTuples.append((tracker,adversaryBB))
            
            #reinit usefulPlayersTrackers
            playersTrackersTuples = []
            for i in range(len(players)):
                x, y, w, h = players[i]
                playerBB = (x, y, w, h)
                tracker = createTracker()
                tracker.init(frame, playerBB)
                playersTrackersTuples.append((tracker,playerBB))
            
            
        cv2.imshow('img',frame)
    if cv2.waitKey(1)==27:
        break
    
cap.release()
cv2.destroyAllWindows()

opened
No lastAdversaryTracker
No usefulPlayersTracker
out
out
out
opened
(409, 88)
opened
(409, 88)
opened
(409, 88)
opened
(408, 89)
opened
(408, 89)
opened
(408, 89)
opened
(408, 89)
opened
(408, 89)
opened
(408, 89)
opened
(408, 89)
opened
(408, 90)
opened
(408, 90)
opened
(408, 90)
opened
(408, 90)
opened
(408, 90)
opened
(408, 90)
opened
(408, 90)
opened
(408, 90)
opened
(407, 90)
opened
lastAdversaryTracker Fail
No usefulPlayersTracker
out
out
out
opened
(407, 90)
opened
(406, 90)
opened
(407, 91)
opened
(405, 90)
opened
(405, 91)
opened
(405, 91)
opened
(404, 91)
opened
(402, 91)
opened
(402, 91)
opened
(402, 91)
opened
(402, 91)
opened
(401, 91)
opened
(399, 91)
opened
(399, 91)
opened
lastAdversaryTracker Fail
lastAdversaryTracker Fail
No usefulPlayersTracker
out
out
out
opened
(396, 92)
opened
(395, 92)
opened
(395, 92)
opened
lastAdversaryTracker Fail
No usefulPlayersTracker
out
out
out
opened
(392, 92)
opened
(392, 92)
opened
(391, 92)
opened
lastAdversaryTracker Fail
No u

opened
(224, 24)
opened
(223, 24)
opened
(223, 23)
opened
(222, 23)
opened
(222, 22)
usefulPlayersTracker Fail
out
out
out
opened
(287, 39)
opened
(285, 39)
opened
(281, 39)
usefulPlayersTracker Fail
out
out
out
opened
(216, 21)
opened
(216, 20)
opened
(214, 21)
opened
(214, 20)
opened
(213, 20)
opened
lastAdversaryTracker Fail
No usefulPlayersTracker
out
out
out
opened
(212, 20)
opened
(211, 20)
opened
(209, 20)
opened
(209, 19)
opened
lastAdversaryTracker Fail
No usefulPlayersTracker
out
out
out
opened
(203, 21)
opened
(202, 21)
opened
(200, 21)
opened
(198, 20)
opened
(197, 20)
opened
lastAdversaryTracker Fail
No usefulPlayersTracker
out
out
out
opened
(193, 20)
opened
(191, 20)
opened
(189, 20)
opened
lastAdversaryTracker Fail
No usefulPlayersTracker
out
out
out
opened
(185, 20)
opened
(183, 20)
opened
(180, 19)
opened
lastAdversaryTracker Fail
No usefulPlayersTracker
out
out
out
opened
(206, 33)
opened
(203, 33)
opened
lastAdversaryTracker Fail
No usefulPlayersTracker
out
out
out


KeyboardInterrupt: 

# Conclusioni
Il software è migliorato molto con l'utilizzo del tracker in sostituzione del solo object detector. Tuttavia ci sono molti difetti che dovrebbero essere corretti:
* La matrice di trasformazione non è precisa al punto da mostrare la linea di fuorigioco sempre nella corretta posizione rispetto al player che stabilisce il fuorigioco. 
* Il giocatore preso in esame come ultimo non risulta essere sempre davvero l'ultimo, poichè se un giocatore rientra nella scena e diventa l'ultimo presente, il tracker continua a tracciare il movimento del giocatore selezionato precedentemente. Per migliorare questo comportamento si potrebbe rieseguire la fase di object detection su base temporale, oltre che quando il giocatore selezionato viene "perso di vista" dal tracker.



Questo è il codice usato per calcolare la permission matrix

In [ ]:
import cv2
import numpy as np

src=cv2.imread('./TransformationMatrix/src.jpg')
dst=cv2.imread('./TransformationMatrix/edit.jpg')

#play with numbers to do the correct transform...
pts1 = np.float32([[0,556],[226,40],[1000,556],[760,40]]) #vera
pts2 = np.float32([[0,556],[0,0],[1000,556],[100,0]]) #finta

pts3 = np.float32([[943,395]])


M = cv2.getPerspectiveTransform(pts1,pts2)
print(M)

pts3o=cv2.perspectiveTransform(pts3[None, :, :],M)


x=int(pts3o[0][0][0])
y=int(pts3o[0][0][1])
p=(x,y)


#cv2.circle(dst,p, 5, (0,0,255),-1)


cv2.imshow('first',dst)
cv2.imshow('second',src)

cv2.waitKey(0)

cv2.destroyAllWindows()